In [1]:
import requests
from bs4 import BeautifulSoup
import random
from tqdm import tqdm
import pandas as pd

URL_BASE = 'https://en.wikipedia.org'
URL_KDRAMAS_LIST = URL_BASE + '/wiki/List_of_South_Korean_dramas'

In [4]:
# get page links
response = requests.get(
	url=URL_KDRAMAS_LIST,
)
soup = BeautifulSoup(response.content, 'html.parser')
df = {
    s: [] for s in ['link', 'title', 'year']
}
for li in soup.find_all("li"):
    itag = li
    if itag.i:
        df['link'].append(itag.a.get('href'))
        df['title'].append(itag.a.get('title').strip())
        df['year'].append(itag.contents[1].strip().strip('()'))
df = pd.DataFrame.from_dict(df)

In [5]:
def check_for_title(h):
    if h.span: # and h.span.has_key('mw-headline'):
        try:
            content = h.span.contents[0].lower()
            if content in ['synopsis', 'plot']:
                text = h.span.find_next('p')
                return text.contents
        except:
            pass
    return False
        
def check_all_titles(soup):
    for h in soup.find_all('h2'):
        text = check_for_title(h)
        if text:
            return text
    return None

df['synopsis'] = None
for i in tqdm(range(df.shape[0])):
    link = df.iloc[i]['link']
    response = requests.get(
        url=URL_BASE + link,
    )
    soup = BeautifulSoup(response.content, 'html.parser')
    df['synopsis'][i] = check_all_titles(soup)

100%|██████████| 1336/1336 [02:31<00:00,  8.85it/s]


In [6]:
# clean up links and refs    
for i in tqdm(range(df.shape[0])):
    syn = df['synopsis'].iloc[i]
    if syn is not None:
        s = []
        for comp in syn:
            if 'Tag' in str(type(comp)):
        
                # reference
                if str(comp).startswith('<sup'): 
                    pass
                # actual link
                else: 
                    try:
                        s.append(str(comp.contents[0]))
                    except:
                        pass
            else:
                s.append(str(comp))
            
        syn = ''.join(s)
    df['synopsis'][i] = syn    
df.to_csv('kdrama_synopses.csv')

100%|██████████| 1336/1336 [00:00<00:00, 1887.38it/s]


In [4]:
synopses = pd.read_csv('kdrama_synopses.csv')['synopsis']

In [7]:
synopses.dropna().to_csv('../titles.csv', index=False)

/accounts/projects/vision/.local/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.
